# Comparison of the most accurate CNN models

In [2]:
###  Add mathematical libraries
import numpy as np
import pandas as pd

# Graphical libraries and items.
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots, draw


In [3]:
# Extract the best model for each max_len


columns = ["timestamp", "max_len", "batch_size", "embed_dim", \
           "num_filters", "kernel_sizes", "dense_layer_dims", \
           "dropout_rate", "val_accuracy_best", "val_accuracy_best_epoch"]


best_models = pd.DataFrame(columns = columns)

for file,desc in [ ("./runs2bal.log",      "CNN withn max_len of 100"), \
                   ("./runs2ballen50.log", "CNN withn max_len of 50"), \
                   ("./runs2ballen21.log", "CNN withn max_len of 21 (Average Sentence Length)"),
                   ("./runs2ballen17.log", "CNN withn max_len of 17 (Median Sentence Length)") ]:
    
    # Load the log file into a dataframe
    df = pd.read_csv(file, sep='|', skiprows=(), header=(0))
    
    metric_cols  = [ f"val_accuracy-{epoch}" for epoch in range(0,20) ]
    
    # Determine the best accuracy and the epoch it was found in for each run
    df["val_accuracy_best"] = df[metric_cols].max(axis=1)
    df["val_accuracy_best_epoch"] = df[metric_cols].idxmax(axis=1).str.extract('(\d+)').astype(int)
    
    best_model = df.sort_values('val_accuracy_best', ascending=False)[:1][columns]
    best_models = best_models.append(best_model, ignore_index=True)
    
best_models.sort_values('val_accuracy_best', ascending=False,)[:10]

,timestamp,max_len,batch_size,embed_dim,num_filters,kernel_sizes,dense_layer_dims,dropout_rate,val_accuracy_best,val_accuracy_best_epoch
1,210409-235328,50,50,50,"[96, 96, 96]","[8, 16, 32]",[8],0.2,0.976492,6
0,210409-210515,100,50,50,"[64, 64, 64]","[4, 8, 16]",[8],0.2,0.975588,8
3,210409-225707,17,50,50,"[96, 96, 96]","[8, 12, 16]",[32],0.2,0.970615,19
2,210410-004542,21,50,50,"[64, 64, 64]","[8, 12, 16]",[8],0.2,0.969259,14


In the discussion, we use the convention M<sub>n</sub> and A<sub>n</sub> to be the model and accuracy for the best model with max_len = n.

So, we observe the following:
1. The A<sub>50</sub> is slightly higher than A<sub>100</sub> by 0.18%
2. Both M<sub>50</sub> and M<sub>100</sub> use the same four kernel sizes, but the M<sub>100</sub> case runs almost 4 times as many filter instances, significantly increasing the complexity of the model.
3. A<sub>21</sub> and A<sub>17</sub> are still over 97% with the validation dataset, so we can look at lower complexity models without sacrificing too much accuracy which is a viable option for detection at the edge.

In [15]:
# Check out the best smaller models, ones with only [8,16] filter sizes

best_small_models = df[df["kernel_sizes"] == "[8, 16]"].sort_values('val_accuracy_best', ascending=False)[:1][columns]
best_small_models

,timestamp,max_len,batch_size,embed_dim,num_filters,kernel_sizes,dense_layer_dims,dropout_rate,val_accuracy_best,val_accuracy_best_epoch
4,210409-222341,17,50,50,"[16, 32]","[8, 16]",[8],0.2,0.960669,7


In [13]:
### Save the best model information for future use

best_models.to_pickle("./best_models.pkl")
best_small_models.to_pickle("./best_small_models.pkl")